In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(0)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
#TEST NOTES:  Replacing batchnorm with instance norm; Adding "shock absorbing" training sessions between size changes.
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'bwc_withattn_sn_small'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id + '_cont')

gpath = IMAGENET.parent/('bwc_withattn_sn_small_gen_192.h5')
dpath = IMAGENET.parent/('bwc_withattn_sn_small_critic_192.h5')

c_lr=1e-3
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/5
g_lrs = np.array([g_lr/100,g_lr/10,g_lr])

keep_pcts=[0.50,0.50]
gen_freeze_tos=[-1,0]

sn=True
self_attention=sn
lrs_unfreeze_factor=0.25

x_tfms = [BlackAndWhiteTransform()]
torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=1, self_attention=self_attention, sn=sn, leakyReLu=False).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
load_model(netG, gpath)

netD = DCCritic(ni=3, nf=128, scale=16, self_attention=self_attention, sn=sn).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
load_model(netD, dpath)

In [5]:
trainer = WGANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e2)], sn=sn)
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=100)

In [6]:
scheds=[]

#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[64,64], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, keep_pcts=keep_pcts, 
#    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))

#unshock
#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[96], bss=[32], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.25], 
#    save_base_name=proj_id, c_lrs=c_lrs/10, g_lrs=g_lrs/10, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[96,96], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, keep_pcts=keep_pcts, 
#    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))


#unshock
#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[128], bss=[32], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.25], 
#    save_base_name=proj_id, c_lrs=c_lrs/10, g_lrs=g_lrs/10, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[128,128], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, keep_pcts=keep_pcts, 
#    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))



#unshock
#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.25], 
#    save_base_name=proj_id, c_lrs=c_lrs/20, g_lrs=g_lrs/20, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))


#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.5], 
#    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))


scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[12], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.5], 
    save_base_name=proj_id, c_lrs=c_lrs/3, g_lrs=g_lrs/3, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0]))


#unshock
scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/40, g_lrs=g_lrs/40, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.5], 
    save_base_name=proj_id, c_lrs=c_lrs/4, g_lrs=g_lrs/4, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[6], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.5], 
    save_base_name=proj_id, c_lrs=c_lrs/4, g_lrs=g_lrs/4, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0]))


In [ ]:
trainer.train(scheds=scheds)

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")
/media/jason/Projects/Deep Learning/Deep Learning Projects/black and white to color/fasterai/generators.py:160: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h<target_h or w<target_w:


Failed to generate graph for model: %43 : Long() = onnx::Constant[value={0}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400
Failed to generate graph for model: %465 : Long() = onnx::Constant[value={2}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400
  0%|          | 0/52352 [00:00<?, ?it/s]

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


  0%|          | 20/52352 [00:12<4:53:46,  2.97it/s]
WDist 1.7413854598999023; RScore 1.223649263381958; FScore 0.5177361369132996; GAddlLoss [2.62856]; Iters: 10; GCost: 0.34572458267211914; GPenalty: [0]; ConPenalty: [0]
  0%|          | 40/52352 [00:18<4:47:07,  3.04it/s]
WDist 1.4601233005523682; RScore 0.9085737466812134; FScore 0.55154949426651; GAddlLoss [2.12212]; Iters: 20; GCost: 0.458745539188385; GPenalty: [0]; ConPenalty: [0]
  0%|          | 60/52352 [00:24<4:40:07,  3.11it/s]
WDist 1.8464860916137695; RScore 1.3413041830062866; FScore 0.5051819682121277; GAddlLoss [2.11998]; Iters: 30; GCost: 0.4131890833377838; GPenalty: [0]; ConPenalty: [0]
  0%|          | 80/52352 [00:30<4:38:36,  3.13it/s]
WDist 2.1628310680389404; RScore 1.374384880065918; FScore 0.7884462475776672; GAddlLoss [2.2769]; Iters: 40; GCost: 0.4855770468711853; GPenalty: [0]; ConPenalty: [0]
  0%|          | 100/52352 [00:37<4:39:22,  3.12it/s]
WDist 1.7742055654525757; RScore 1.0658165216445923; FScore